In [ ]:
!pip install boto3

In [ ]:
!cp -r /kaggle/input/cable/* /kaggle/working

In [ ]:
import boto3
import uuid, os
from datetime import date

current_date = date.today()
date_name = current_date.strftime("%Y-%m-%d")

ACCESS_KEY = os.environ.get('AWS_ACCESS_KEY_ID')
SECRET_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
BUCKET_NAME = os.environ.get('BUCKET_NAME')
DATASET_NAME = "cycle-gan/cable"
CABLE_NAME = "weights.zip"
CABLE_PATH = f"{DATASET_NAME}/cable_weights-{date_name}.zip"

session = boto3.Session(
    aws_access_key_id=ACCESS_KEY, 
    aws_secret_access_key=SECRET_KEY
)

s3 = session.resource('s3')
bucket = s3.Bucket(BUCKET_NAME)

# Introduction

In this project, we will attempt to greate a Cycle Generative Adversarial Network (cycleGAN) in order to manipulate defective industrial parts. We will attempt to augment images data.

Our first step is to load the neccessary libraries we will be using for the entire process.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as data
import random
from torchvision import transforms
from PIL import Image
import os , itertools
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

Since we are using two platforms, we will be assigning a directory to the location of the data for each platform. This will make it easier to access the data, depending the platform we will be using.

The two platforms will be:
1. A local installation of  Jupyter Notebook - work on local codes. GPU is limited
2. A Google Colaboratory Notebook - Use of available resources and GPU to run the model

In [ ]:
    data_dir = 'cable'
    train_dir = os.path.join(data_dir,'train')
    test_dir = os.path.join(data_dir,'test')

## Parameters

In this block, we will create global parameters which we can use at different areas of this project.

In [ ]:
# Creating the Parameters

parameters = {
    'batch_size': 1,
    'input_size': 256,
    'resize_scale': 286,
    'crop_size': 256,
    'fliplr':True,
    
    # Model parameters
    'num_epochs': 100,
    'decay_epoch': 100,
    'ngf': 32,   # Number of generator filters
    'ndf': 64,   # Number of discriminator filters
    'num_resnet': 6, # Number of resnet blocks
    'lrG': 0.0002,    # Learning rate for generator
    'lrD': 0.0002,    # Learning rate for discriminator
    'beta1': 0.5 ,    # Beta1 for Adam optimizer
    'beta2': 0.999 ,  # Beta2 for Adam optimizer
    'lambdaA': 10 ,   # LambdaA for cycle loss
    'lambdaB': 10  ,  # LambdaB for cycle loss
}
          

## Creating functions

In the next set, we will create several functions that will be used throughout. The first function will convert our data into Numpy, which we can call at a later point.

We will also set a variable so we can use a GPU for faster processing.

In [ ]:
# Convert to Numpy 
def to_numpy(x):
    return x.data.cpu().numpy()

# Use GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [1]:
!mkdir training_images

### Transformer

Since we will be using a __transformer__ at a later stage, we will set a transformer variable and assign it procedures which transforms an image size accordingly. This transformer will also convert the image into a Tensor and Normalize it as well.

In [ ]:
transform = transforms.Compose([
    transforms.Resize(size=params['input_size']),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])


Defining a function which will __plot__ the result from training

In [ ]:
def plot_train_result(real_image, gen_image, recon_image, epoch, save=False,  show=False, fig_size=(15, 15)):
    fig, axes = plt.subplots(2, 3, figsize=fig_size)
    imgs = [to_np(real_image[0]), to_np(gen_image[0]), to_np(recon_image[0]),
            to_np(real_image[1]), to_np(gen_image[1]), to_np(recon_image[1])]
    for ax, img in zip(axes.flatten(), imgs):
        ax.axis('off')
        #ax.set_adjustable('box-forced')
        # Scale to 0-255
        img = img.squeeze()
        img = (((img - img.min()) * 255) / (img.max() - img.min())).transpose(1, 2, 0).astype(np.uint8)
        ax.imshow(img, cmap=None, aspect='equal')
    plt.subplots_adjust(wspace=0, hspace=0)

    title = 'Epoch {0}'.format(epoch + 1)
    fig.text(0.5, 0.04, title, ha='center')

    # save figure
    if save:
        save_fn = f'Result_epoch_{epoch+1}.png'
        plt.savefig(save_fn)

    if show:
        plt.show()
    else:
        plt.close()

Next, we will define an __ImagePool__ function

In [ ]:
class ImagePool():
    def __init__(self, pool_size):
        self.pool_size = pool_size
        if self.pool_size > 0:
            self.num_imgs = 0
            self.images = []

    def query(self, images):
        if self.pool_size == 0:
            return images
        return_images = []
        for image in images.data:
            image = torch.unsqueeze(image, 0)
            if self.num_imgs < self.pool_size:
                self.num_imgs = self.num_imgs + 1
                self.images.append(image)
                return_images.append(image)
            else:
                p = random.uniform(0, 1)
                if p > 0.5:
                    random_id = random.randint(0, self.pool_size-1)
                    tmp = self.images[random_id].clone()
                    self.images[random_id] = image
                    return_images.append(tmp)
                else:
                    return_images.append(image)
        return_images = Variable(torch.cat(return_images, 0))
        return return_images

### Creating the Dataloader

In this section, we begin by creating the __dataloader__ which we will use to retrieve the data in parts.

In [ ]:
class DatasetFromFolder(data.Dataset):
    def __init__(self, image_dir, subfolder='train', transform=None, resize_scale=None, crop_size=None, fliplr=False):
        super(DatasetFromFolder, self).__init__()
        self.input_path = os.path.join(image_dir, subfolder)
        self.image_filenames = [x for x in sorted(os.listdir(self.input_path))]
        self.transform = transform

        self.resize_scale = resize_scale
        self.crop_size = crop_size
        self.fliplr = fliplr

    def __getitem__(self, index):
        # Load Image
        img_fn = os.path.join(self.input_path, self.image_filenames[index])
        img = Image.open(img_fn).convert('RGB')

        # preprocessing
        if self.resize_scale:
            img = img.resize((self.resize_scale, self.resize_scale), Image.BILINEAR)

        if self.crop_size:
            x = random.randint(0, self.resize_scale - self.crop_size + 1)
            y = random.randint(0, self.resize_scale - self.crop_size + 1)
            img = img.crop((x, y, x + self.crop_size, y + self.crop_size))
        if self.fliplr:
            if random.random() < 0.5:
                img = img.transpose(Image.FLIP_LEFT_RIGHT)

        if self.transform is not None:
            img = self.transform(img)

        return img

    def __len__(self):
        return len(self.image_filenames)

### Creating the Convolutional Block

The __convolution__ block will consist of two things.

1. Initialization

2. Forward pass.

In [ ]:
class ConvBlock(torch.nn.Module):
    def __init__(self,input_size,output_size,kernel_size=3,stride=2,padding=1,activation='relu',batch_norm=True):
        super(ConvBlock,self).__init__()
        self.conv = torch.nn.Conv2d(input_size,output_size,kernel_size,stride,padding)
        self.batch_norm = batch_norm
        self.bn = torch.nn.InstanceNorm2d(output_size)
        self.activation = activation
        self.relu = torch.nn.ReLU(True)
        self.lrelu = torch.nn.LeakyReLU(0.2,True)
        self.tanh = torch.nn.Tanh()
    def forward(self,x):
        if self.batch_norm:
            out = self.bn(self.conv(x))
        else:
            out = self.conv(x)

        if self.activation == 'relu':
            return self.relu(out)
        elif self.activation == 'lrelu':
            return self.lrelu(out)
        elif self.activation == 'tanh':
            return self.tanh(out)
        elif self.activation == 'no_act':
            return out


### De-Convolution Function

In this class we will create, we will need to bring back the convolution from the results previously processed. There will be two functions inside this class.

1. Initialization

2. Forward pass

In [ ]:
class DeconvBlock(torch.nn.Module):
    def __init__(self,input_size,output_size,kernel_size=3,stride=2,padding=1,output_padding=1,activation='relu',batch_norm=True):
        super(DeconvBlock,self).__init__()
        self.deconv = torch.nn.ConvTranspose2d(input_size,output_size,kernel_size,stride,padding,output_padding)
        self.batch_norm = batch_norm
        self.bn = torch.nn.InstanceNorm2d(output_size)
        self.activation = activation
        self.relu = torch.nn.ReLU(True)
    def forward(self,x):
        if self.batch_norm:
            out = self.bn(self.deconv(x))
        else:
            out = self.deconv(x)
        if self.activation == 'relu':
            return self.relu(out)
        elif self.activation == 'lrelu':
            return self.lrelu(out)
        elif self.activation == 'tanh':
            return self.tanh(out)
        elif self.activation == 'no_act':
            return out


### Residual Learning Block

In this secton, we will create a __residual learning__ block or __Resnet__. This class will also contain two functions.

1. Initialization

2. Forward pass

In [ ]:
class ResnetBlock(torch.nn.Module):
    def __init__(self,num_filter,kernel_size=3,stride=1,padding=0):
        super(ResnetBlock,self).__init__()
        conv1 = torch.nn.Conv2d(num_filter,num_filter,kernel_size,stride,padding)
        conv2 = torch.nn.Conv2d(num_filter,num_filter,kernel_size,stride,padding)
        bn = torch.nn.InstanceNorm2d(num_filter)
        relu = torch.nn.ReLU(True)
        pad = torch.nn.ReflectionPad2d(1)

        self.resnet_block = torch.nn.Sequential(pad, conv1, bn, relu, pad, conv2, bn)
    def forward(self,x):
        out = self.resnet_block(x)
        return out

## Creating the Generator

We will now create the __Generator__. The Generator will consist of the following:
1. Encoder
2. Transformer
3. Decoder

In addition, we will also be adding __weights__.

After creating the class, we will create the actuall Generator.

In [ ]:
class Generator(torch.nn.Module):
    def __init__(self,input_dim,num_filter,output_dim,num_resnet):
        super(Generator,self).__init__()

        # Reflection padding
        self.pad = torch.nn.ReflectionPad2d(3)
        # Encoder
        self.conv1 = ConvBlock(input_dim,num_filter,kernel_size=7,stride=1,padding=0)
        self.conv2 = ConvBlock(num_filter,num_filter*2)
        self.conv3 = ConvBlock(num_filter*2,num_filter*4)
        # Resnet blocks
        self.resnet_blocks = []
        for i in range(num_resnet):
            self.resnet_blocks.append(ResnetBlock(num_filter*4))
        self.resnet_blocks = torch.nn.Sequential(*self.resnet_blocks)
        #Decoder
        self.deconv1 = DeconvBlock(num_filter*4,num_filter*2)
        self.deconv2 = DeconvBlock(num_filter*2,num_filter)
        self.deconv3 = ConvBlock(num_filter,output_dim,kernel_size=7,stride=1,padding=0,activation='tanh',batch_norm=False)

    def forward(self,x):
        # Encoder
        enc1 = self.conv1(self.pad(x))
        enc2 = self.conv2(enc1)
        enc3 = self.conv3(enc2)
        # Resnet blocks
        res = self.resnet_blocks(enc3)
        # Decoder
        dec1 = self.deconv1(res)
        dec2 = self.deconv2(dec1)
        out = self.deconv3(self.pad(dec2))
        return out

    def normal_weight_init(self,mean=0.0,std=0.02):
        for m in self.children():
            if isinstance(m,ConvBlock):
                torch.nn.init.normal_(m.conv.weight,mean,std)
            if isinstance(m,DeconvBlock):
                torch.nn.init.normal_(m.deconv.weight,mean,std)
            if isinstance(m,ResnetBlock):
                torch.nn.init.normal_(m.conv.weight,mean,std)
                torch.nn.init.constant_(m.conv.bias,0)


### Creating the Generator and Testing the Neural Network

Since we will be using a Cycle GAN, which requires 2 Neural Networks competing with each other, we will create 2 Generators as its requirement. We will also test it by calling it and view the results created earlier.

#### Generator A

In [ ]:
G_A = Generator(3, parameters['ngf'], 3, parameters['num_resnet']).cuda() # input_dim, num_filter, output_dim, num_resnet
G_A.normal_weight_init(mean=0.0, std=0.02)
G_A

#### Generator B

Repeating the previous step, but for Generator B.

In [ ]:
G_B = Generator(3, parameters['ngf'], 3, parameters['num_resnet']).cuda()
G_B.normal_weight_init(mean=0.0, std=0.02)
G_B

## Creating the Discriminator

The Discriminator will consist of the following
1. Initialization
2. Forward Pass
3. Weights

After creating the class, we will create the actuall Discriminator.


In [ ]:
class Discriminator(torch.nn.Module):
    def __init__(self,input_dim,num_filter,output_dim):
        super(Discriminator,self).__init__()
        conv1 = ConvBlock(input_dim,num_filter,kernel_size=4,stride=2,padding=1,activation='lrelu',batch_norm=False)
        conv2 = ConvBlock(num_filter,num_filter*2,kernel_size=4,stride=2,padding=1,activation='lrelu')
        conv3 = ConvBlock(num_filter*2,num_filter*4,kernel_size=4,stride=2,padding=1,activation='lrelu')
        conv4 = ConvBlock(num_filter*4,num_filter*8,kernel_size=4,stride=1,padding=1,activation='lrelu')
        conv5 = ConvBlock(num_filter*8,output_dim,kernel_size=4,stride=1,padding=1,activation='no_act',batch_norm=False)
        self.conv_blocks = torch.nn.Sequential(conv1, conv2, conv3, conv4, conv5)
    
    def forward(self,x):
        out = self.conv_blocks(x)
        return out

    def normal_weight_init(self,mean=0.0,std=0.02):
        for m in self.children():
            if isinstance(m,ConvBlock):
                torch.nn.init.normal_(m.conv.weight.data,mean,std)

### Creating the Discriminator and Testing the Neural Network

Similar with the Generator, which requires 2 Neural Networks. A cycleGAN also uses 2 Discriminators.

#### Discriminator A

In [ ]:
# Creating the first Discriminator
D_A = Discriminator(3, parameters['ndf'], 1).cuda() # input_dim, num_filter, output_dim
D_A.normal_weight_init(mean=0.0, std=0.02)
D_A

#### Discriminator B

In [ ]:
# Creating the second Discriminator
D_B = Discriminator(3, parameters['ndf'], 1).cuda()
D_B.normal_weight_init(mean=0.0, std=0.02)
D_B

## Training Block / DataLoaders

In this block, we will create the actual training block we will be using to train our model.

In [ ]:
train_data_A = DatasetFromFolder(train_dir, subfolder='good', transform=transform,resize_scale=parameters['resize_scale'], crop_size=parameters['crop_size'], fliplr=parameters['fliplr'])
train_data_loader_A = torch.utils.data.DataLoader(dataset=train_data_A,batch_size=parameters['batch_size'], shuffle=True)
train_data_B = DatasetFromFolder(data_dir, subfolder='bad', transform=transform, resize_scale=parameters['resize_scale'], crop_size=parameters['crop_size'], fliplr=parameters['fliplr'])
train_data_loader_B = torch.utils.data.DataLoader(dataset=train_data_B, batch_size=parameters['batch_size'], shuffle=True)

#Load test data
test_data_A = DatasetFromFolder(test_dir, subfolder='good', transform=transform)
test_data_loader_A = torch.utils.data.DataLoader(dataset=test_data_A, batch_size=parameters['batch_size'], shuffle=False)
test_data_B = DatasetFromFolder(test_dir, subfolder='bad', transform=transform)
test_data_loader_B = torch.utils.data.DataLoader(dataset=test_data_B, batch_size=parameters['batch_size'], shuffle=False)

## Creating the Optimizer

In this block, we need to create the learning optimizers. We will have one optimizer for the Generator and two optimizers for the Discriminator. The Discriminator has two optimizers as this function checks the data generated by the Generator.

In [ ]:
G_optimizer = torch.optim.Adam(itertools.chain(G_A.parameters(), G_B.parameters()), lr=parameters['lrG'], betas=(parameters['beta1'], parameters['beta2']))
D_A_optimizer = torch.optim.Adam(D_A.parameters(), lr=parameters['lrD'], betas=(parameters['beta1'], parameters['beta2']))
D_B_optimizer = torch.optim.Adam(D_B.parameters(), lr=parameters['lrD'], betas=(parameters['beta1'], parameters['beta2']))

## Defining Losses

In order to make sure that there is learning, we will create cycle losses, which we will pass on at a later time.

In [ ]:
MSE_Loss = torch.nn.MSELoss().cuda()
L1_Loss = torch.nn.L1Loss().cuda()

In [ ]:
D_A_avg_losses = []
D_B_avg_losses = []
G_A_avg_losses = []
G_B_avg_losses = []
cycle_A_avg_losses = []
cycle_B_avg_losses = []

## Training the model and retreiving Images



Using the image pool created earliler, we will create a pool of variables which we will be using for training at the next cell block.

In [1]:
num_pool = 50
fake_A_pool = ImagePool(num_pool)
fake_B_pool = ImagePool(num_pool)

# Get specific test images
test_real_A_data = train_data_A.__getitem__(8).unsqueeze(0)
test_real_B_data = train_data_B.__getitem__(6).unsqueeze(0)

step = 0

NameError: name 'ImagePool' is not defined

We will now run the Training sequence in the block. It is also in these block that learning begins and the time to process the results can take several hours. 

There will be two results displayed below.
1. The epoch number where the learning takes place. This will also include the loss.
2. The image results on how well it did.

In [ ]:
for epoch in range(parameters['num_epochs']):
    D_A_losses = []
    D_B_losses = []
    G_A_losses = []
    G_B_losses = []
    cycle_A_losses = []
    cycle_B_losses = []

    # Learing rate decay
    if(epoch + 1) > parameters['decay_epoch']:
        D_A_optimizer.param_groups[0]['lr'] -= parameters['lrD'] / (parameters['num_epochs'] - parameters['decay_epoch'])
        D_B_optimizer.param_groups[0]['lr'] -= parameters['lrD'] / (parameters['num_epochs'] - parameters['decay_epoch'])
        G_optimizer.param_groups[0]['lr'] -= parameters['lrG'] / (parameters['num_epochs'] - parameters['decay_epoch'])


    # training
    for i, (real_A, real_B) in enumerate(zip(train_data_loader_A, train_data_loader_B)):

        # input image data
        real_A = real_A.to(device)
        real_B = real_B.to(device)

        # -------------------------- train generator G --------------------------
        # A --> B
        fake_B = G_A(real_A)
        D_B_fake_decision = D_B(fake_B)
        G_A_loss = MSE_Loss(D_B_fake_decision, Variable(torch.ones(D_B_fake_decision.size()).cuda()))

        # forward cycle loss
        recon_A = G_B(fake_B)
        cycle_A_loss = L1_Loss(recon_A, real_A) * parameters['lambdaA']

        # B --> A
        fake_A = G_B(real_B)
        D_A_fake_decision = D_A(fake_A)
        G_B_loss = MSE_Loss(D_A_fake_decision, Variable(torch.ones(D_A_fake_decision.size()).cuda()))

        # backward cycle loss
        recon_B = G_A(fake_A)
        cycle_B_loss = L1_Loss(recon_B, real_B) * parameters['lambdaB']

        # Back propagation
        G_loss = G_A_loss + G_B_loss + cycle_A_loss + cycle_B_loss
        G_optimizer.zero_grad()
        G_loss.backward()
        G_optimizer.step()


        # -------------------------- train discriminator D_A --------------------------
        D_A_real_decision = D_A(real_A)
        D_A_real_loss = MSE_Loss(D_A_real_decision, Variable(torch.ones(D_A_real_decision.size()).cuda()))

        fake_A = fake_A_pool.query(fake_A)

        D_A_fake_decision = D_A(fake_A)
        D_A_fake_loss = MSE_Loss(D_A_fake_decision, Variable(torch.zeros(D_A_fake_decision.size()).cuda()))

        # Back propagation
        D_A_loss = (D_A_real_loss + D_A_fake_loss) * 0.5
        D_A_optimizer.zero_grad()
        D_A_loss.backward()
        D_A_optimizer.step()

        # -------------------------- train discriminator D_B --------------------------
        D_B_real_decision = D_B(real_B)
        D_B_real_loss = MSE_Loss(D_B_real_decision, Variable(torch.ones(D_B_fake_decision.size()).cuda()))

        fake_B = fake_B_pool.query(fake_B)

        D_B_fake_decision = D_B(fake_B)
        D_B_fake_loss = MSE_Loss(D_B_fake_decision, Variable(torch.zeros(D_B_fake_decision.size()).cuda()))

        # Back propagation
        D_B_loss = (D_B_real_loss + D_B_fake_loss) * 0.5
        D_B_optimizer.zero_grad()
        D_B_loss.backward()
        D_B_optimizer.step()

        # ------------------------ Print -----------------------------
        # loss values
        D_A_losses.append(D_A_loss.item())
        D_B_losses.append(D_B_loss.item())
        G_A_losses.append(G_A_loss.item())
        G_B_losses.append(G_B_loss.item())
        cycle_A_losses.append(cycle_A_loss.item())
        cycle_B_losses.append(cycle_B_loss.item())

        if i % 100 == 0:
            print('Epoch [%d/%d], Step [%d/%d], D_A_loss: %.4f, D_B_loss: %.4f, G_A_loss: %.4f, G_B_loss: %.4f'
                  % (epoch+1, parameters['num_epochs'], i+1, len(train_data_loader_A), D_A_loss.item(), D_B_loss.item(), G_A_loss.item(), G_B_loss.item()))

        step += 1

    D_A_avg_loss = torch.mean(torch.FloatTensor(D_A_losses))
    D_B_avg_loss = torch.mean(torch.FloatTensor(D_B_losses))
    G_A_avg_loss = torch.mean(torch.FloatTensor(G_A_losses))
    G_B_avg_loss = torch.mean(torch.FloatTensor(G_B_losses))
    cycle_A_avg_loss = torch.mean(torch.FloatTensor(cycle_A_losses))
    cycle_B_avg_loss = torch.mean(torch.FloatTensor(cycle_B_losses))

    # avg loss values for plot
    D_A_avg_losses.append(D_A_avg_loss.item())
    D_B_avg_losses.append(D_B_avg_loss.item())
    G_A_avg_losses.append(G_A_avg_loss.item())
    G_B_avg_losses.append(G_B_avg_loss.item())
    cycle_A_avg_losses.append(cycle_A_avg_loss.item())
    cycle_B_avg_losses.append(cycle_B_avg_loss.item())

    # Show result for test image
    test_real_A = test_real_A_data.cuda()
    test_fake_B = G_A(test_real_A)
    test_recon_A = G_B(test_fake_B)

    test_real_B = test_real_B_data.cuda()
    test_fake_A = G_B(test_real_B)
    test_recon_B = G_A(test_fake_A)

    plot_train_result([test_real_A, test_real_B], [test_fake_B, test_fake_A], [test_recon_A, test_recon_B], epoch, save=True)

## Retreiving the Loss results

We will not retreive the losses generated after learning and save them to a csv file.

In [ ]:
all_losses = pd.DataFrame()
all_losses['D_A_avg_losses'] = D_A_avg_losses
all_losses['D_B_avg_losses'] = D_B_avg_losses
all_losses['G_A_avg_losses'] = G_A_avg_losses
all_losses['G_B_avg_losses'] = G_B_avg_losses
all_losses['cycle_A_avg_losses'] = cycle_A_avg_losses
all_losses['cycle_B_avg_losses'] = cycle_B_avg_losses
all_losses.to_csv('avg_losses',index=False)

In [ ]:
all_losses

In [ ]:
#save to csv
all_losses.to_csv('all_losses.csv', index=False)

## Plotting all the Losses

In [ ]:
losses = pd.read_csv('all_losses.csv')

plt.figure(figsize=(20,20))
losses.plot()
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', ncol=1)
plt.show()

## Saving the Models to a file

In order to reproduce, use the same model at a later point. We will save the models state_dict created earlier.

In [ ]:
# Save Generators
torch.save(Generator_A.state_dict(), 'Generator_A.pth')
torch.save(Generator_B.state_dict(), 'Generator_B.pth')

# Save Descriminators
torch.save(Discriminator_A.state_dict(), 'Discriminator_A.pth')
torch.save(Discriminator_B.state_dict(), 'Discriminator_B.pth')

# Save Optimizers
torch.save(Generator_optimizer.state_dict(), 'Generator_optimizer.pth')
torch.save(Discriminator_A_optimizer.state_dict(), 'Discriminator_A_optimizer.pth')
torch.save(Discriminator_B_optimizer.state_dict(), 'Discriminator_B_optimizer.pth')

In [ ]:
zip -r weights.zip .

In [ ]:
try:
    bucket.upload_file(Key=CABLE_PATH, Filename=CABLE_NAME)
    print(f'File {CABLE_NAME} uploaded to S3 bucket {BUCKET_NAME}')
except:
    print("Something went wrong")